In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import re

# --- KONFIGURATION: DATENQUELLEN (GITHUB) ---
base_url = "https://raw.githubusercontent.com/ichzann/DIS08_Data_modeling_7er_Gruppe/main/Daten_sets/blaulicht_scraping/"
#base_url = "https://github.com/ichzann/DIS08_Data_modeling_7er_Gruppe/tree/final_cleanUp/Data/raw/blaulicht_scraping/"
# Liste der Dateien im Repo 
folders_csv_in_repo = [
    "Chemnitz_blaulicht_scrape_2025-12-07.csv",
    "Dortmund_blaulicht_scrape_2025-12-07.csv",
    "Erfurt_blaulicht_scrape_2025-12-09.csv",
    "Nuernberg_blaulicht_scrape_2025-12-09.csv",
    "Saarbruecken_blaulicht_scrape_2025-12-09.csv",
    "München_blaulicht_scrape_2025-12-18.csv"
]

all_docs = []
doc_metadata = []

print("Lade Daten von GitHub und bereite vor...")

# --- SCHRITT 1: DATEN LADEN (Ihsan teil) ---
for file_name in folders_csv_in_repo:
    # URL zusammenbauen
    url = f"{base_url}{file_name}"

    
    try:
        # Pandas kann CSVs direkt von URLs lesen
        df = pd.read_csv(url, on_bad_lines='skip') 
        
        # Spalten bereinigen
        df.columns = [c.lower() for c in df.columns]
        
        # 1. Text kombinieren (Title + Abstract)
        # Wir prüfen sicherheitshalber, welche Spalten da sind
        if 'title' in df.columns and 'abstract' in df.columns:
            df['text'] = df['title'].astype(str) + " " + df['abstract'].astype(str)
        else:
            # Fallback: Alles verbinden, falls Spaltennamen anders sind
            df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        
        # 2. Stadt aus Dateinamen extrahieren
        # Split am Unterstrich (z.B. "Chemnitz_...")
        stadt_name = file_name.split('_')[0]
        
        # 3. Jahr aus Datum extrahieren
        # Wir nutzen hier die robuste Regex-Methode, da sie Format-Unabhängig ist
        if 'datum' in df.columns:
            df['jahr'] = df['datum'].astype(str).str.extract(r'(\d{4})')
        else:
            df['jahr'] = 'Unbekannt'
        
        # Daten sammeln für die Analyse
        # Wir iterieren durch den DataFrame und speichern Text + Metadaten
        for text, jahr in zip(df['text'], df['jahr']):
            all_docs.append(text)
            doc_metadata.append({
                'Stadt': stadt_name, 
                'Jahr': jahr,
                'Original_Datei': file_name
            })
            
        print(f"OK: {file_name} geladen.")

    except Exception as e:
        print(f"FEHLER bei {file_name}: {e}")

# --- SCHRITT 2: ANALYSE (TF-IDF) ---
if not all_docs:
    print("Keine Datengeladen.")
else:
    print(f"\n{len(all_docs)} Dokumente insgesamt. TF-IDF Analyse")

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_docs)
    feature_names = vectorizer.get_feature_names_out()

    #Keyword-Liste
    suchbegriffe = ['drogen', 'kokain', 'koks', 'droge', 'mdma', 'methamphetamin', 'methamphetamine']
    
    # Indizes der Keywords im Vokabular finden
    target_indices = [np.where(feature_names == k)[0][0] for k in suchbegriffe if k in feature_names]

    if target_indices:
        # Summe der Scores berechnen
        scores = np.asarray(tfidf_matrix[:, target_indices].sum(axis=1)).flatten()
        
        # Ergebnisse verknüpfen
        ergebnis_liste = []
        for i, meta in enumerate(doc_metadata):
            ergebnis_liste.append({
                'Stadt': meta['Stadt'],
                'Jahr': meta['Jahr'],
                'Score': scores[i]
            })
        
        ergebnis_df = pd.DataFrame(ergebnis_liste)
        
        # Ungültige Jahre entfernen
        ergebnis_df = ergebnis_df.dropna(subset=['Jahr'])
        
        # --- SCHRITT 3: AGGREGATION & DIFFERENZEN ---
        
        # Gruppieren nach Stadt und Jahr
        zusammenfassung = ergebnis_df.groupby(['Stadt', 'Jahr']).agg(
            Gesamt_Berichte=('Score', 'count'),
            Relevante_Berichte=('Score', lambda x: (x > 0).sum()), 
            Durchschnitts_Score=('Score', 'mean')                       ####Raus score nicht nötig 
        ).reset_index()
        
        # Frequenz berechnen
        zusammenfassung['Frequenz_Prozent'] = (zusammenfassung['Relevante_Berichte'] / zusammenfassung['Gesamt_Berichte']) * 100
        
        #Sortieren (Stadt -> Jahr), damit die Differenzberechnung stimmt
        zusammenfassung = zusammenfassung.sort_values(by=['Stadt', 'Jahr'])

        # Differenzen berechnen Veränderung zum Vorjahr innerhalb der gleichen Stadt
        zusammenfassung['Diff_Score'] = zusammenfassung.groupby('Stadt')['Durchschnitts_Score'].diff()      ####Raus score nicht nötig 
        zusammenfassung['Diff_Frequenz'] = zusammenfassung.groupby('Stadt')['Frequenz_Prozent'].diff()

        # Runden
        zusammenfassung = zusammenfassung.round(4)
        
        # Ausgabe im Terminal
        print("\n--- Analyse-Ergebnis ---")
        print(zusammenfassung.to_string()) 
        
        # Speichern
        output_filename = 'drogen_analyse_github_daten.csv'
        zusammenfassungg.to_csv(output_filename, index=False, sep=';', decimal=',')
        print(f"\nDatei erfolgreich gespeichert: {output_filename}")

    else:
        print("Keine der Suchbegriffe im Textkorpus gefunden.")

Lade Daten von GitHub und bereite vor...
OK: Chemnitz_blaulicht_scrape_2025-12-07.csv geladen.
OK: Dortmund_blaulicht_scrape_2025-12-07.csv geladen.
OK: Erfurt_blaulicht_scrape_2025-12-09.csv geladen.
OK: Nuernberg_blaulicht_scrape_2025-12-09.csv geladen.
OK: Saarbruecken_blaulicht_scrape_2025-12-09.csv geladen.
FEHLER bei München_blaulicht_scrape_2025-12-18.csv: 'ascii' codec can't encode character '\xfc' in position 80: ordinal not in range(128)

21223 Dokumente insgesamt. TF-IDF Analyse

--- Analyse-Ergebnis ---
           Stadt  Jahr  Gesamt_Berichte  Relevante_Berichte  Durchschnitts_Score  Frequenz_Prozent  Diff_Score  Diff_Frequenz
0       Chemnitz  2018                1                   0               0.0000            0.0000         NaN            NaN
1       Chemnitz  2019               15                   0               0.0000            0.0000      0.0000         0.0000
2       Chemnitz  2020               56                   2               0.0045            3.5714   

NameError: name 'zusammenfassungg' is not defined